In [ ]:
from utils import import_dataset_fct
import numpy as np
import pandas as pd
import random

random.seed(42)

In [ ]:
import_dataset_fct.return_filenames()

In [ ]:
df = import_dataset_fct.get_file("Dataset_1_train_asset.csv")
df

In [ ]:
def col_numeric_names(df):
    df = df.rename(columns={x:y for x,y in zip(df.columns,range(0,len(df.columns)))})
    return df

def absolute_growth(df):
    len_df = df.shape[1]
    temp = df.iloc[:,1:len_df]
    temp = col_numeric_names(temp)
    temp2 = df.iloc[:,:len_df-1]
    df2 = temp-temp2
    df2.insert(0, "0", 0)
    df2 = col_numeric_names(df2)
    return df2
df_absolute_growth = absolute_growth(df)
df_absolute_growth

In [ ]:
def standardize(S):
    mean = S.mean(axis=1)
    std = S.std(axis=1)
    keys = np.arange(0, 30, 1)
    S_std = pd.concat([std]*len(keys), keys=keys, axis=1)
    S_mean = pd.concat([mean]*len(keys), keys=keys, axis=1)
    S_stand = (S - S_mean)/S_std
    #S_stand = S/S_std
    return S_stand

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Set up device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def col_numeric_names(df):
    df = df.rename(columns={x:y for x,y in zip(df.columns,range(0,len(df.columns)))})
    return df

def absolute_growth(df):
    len_df = df.shape[1]
    temp = df.iloc[:,1:len_df]
    temp = col_numeric_names(temp)
    temp2 = df.iloc[:,:len_df-1]
    df2 = temp-temp2
    df2.insert(0, "0", 0)
    df2 = col_numeric_names(df2)
    return df2

def get_train_val(url_data, url_payoff, split_percent=0.8):
    df = import_dataset_fct.get_file(url_data)
    pay_off = pd.read_csv(url_payoff, header=None)
    df_absolute_growth = absolute_growth(df)
    df = standardize(df.iloc[:,:30])
    data = np.array(df.values.astype('float32'))
    #scaler = MinMaxScaler(feature_range=(0, 1))
    #data = scaler.fit_transform(data).flatten()
    pay_off = np.array(pay_off)
    n = len(data)
    # Point for splitting data into train and test
    split = int(n*split_percent)
    train_data = data[range(split)]
    val_data = data[split:]
    n = len(df)
    split = int(n*split_percent)
    train_payoff = pay_off[range(split)]
    train_growth = np.array(df_absolute_growth.iloc[0:split, 1:])
    #train_growth = np.array(df_absolute_growth.iloc[0:split, :30])
    val_payoff = pay_off[split:]
    val_growth = np.array(df_absolute_growth.iloc[split:, 1:])
    #val_growth = np.array(df_absolute_growth.iloc[split:, :30])
    return train_data, val_data, train_payoff, val_payoff, train_growth, val_growth, data

url_payoff = "/home/jovyan/hfactory_magic_folders/natixis_data_challenge_22_23/erm/Dataset_1_train_payoff.csv"
url_data = "Dataset_1_train_asset.csv"
train_data, val_data, train_payoff, val_payoff, train_growth, val_growth, data = get_train_val(url_data, url_payoff, split_percent=0.8)

In [ ]:
#print(list(torch.utils.data.TensorDataset(torch.Tensor(train_growth), torch.Tensor(train_payoff))))

In [ ]:
def prepare_data(train_data, val_data, train_payoff, val_payoff, train_growth, val_growth, batch_size):
    train_payoff = torch.Tensor(train_payoff)
    val_payoff = torch.Tensor(val_payoff)
    train_growth = torch.Tensor(train_growth)
    val_growth = torch.Tensor(val_growth)
    train_X = torch.Tensor(train_data)
    train_transaction_costs = torch.Tensor([0.0] * train_X.shape[0])
    val_X = torch.Tensor(val_data)
    val_transaction_costs = torch.Tensor([0.0] * val_X.shape[0])
    dataset_train = torch.utils.data.TensorDataset(train_X, train_payoff, train_growth, train_transaction_costs)
    dataset_val = torch.utils.data.TensorDataset(val_X, val_payoff, val_growth, val_transaction_costs)
    train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, shuffle=False)
    return dataset_train, dataset_val, train_loader, val_loader

batch_size = 256
dataset_train, dataset_val, train_loader, val_loader = prepare_data(train_data, val_data, train_payoff, val_payoff, train_growth, val_growth, batch_size)

In [ ]:
class DeepHedging_RNN(nn.Module):
    def __init__(self, input_size, HL_size, output_size):
        super(DeepHedging_RNN, self).__init__()
        self.rnn = torch.nn.RNN(input_size=input_size,
                                hidden_size=HL_size,
                                num_layers=1)
        self.linear = torch.nn.Linear(HL_size, output_size)
        #self.sigmoid = torch.nn.Sigmoid()
    def forward(self, S):
        out, _ = self.rnn(S)
        out = self.linear(out)
        #out = self.sigmoid(out)
        return out.squeeze()

In [ ]:
import math
import torch.nn.functional as F
'''
class BS_Loss(torch.nn.Module):
    def __init__(self, batch_size, beta=1):
        super(BS_Loss, self).__init__()
        self.beta = beta
        # initialize alpha
        self.q1 = nn.Parameter(torch.randn(1), requires_grad=True)
        self.q2 = nn.Parameter(torch.randn(1), requires_grad=True)
        
    def forward(self, deltas, pay_off, growth, costs):
        S_delta_T = torch.transpose(growth, 0, 1)
        delta_S = torch.diagonal(torch.matmul(deltas, S_delta_T))
        loss = torch.sub(pay_off.squeeze(), delta_S)
        #es_50 = torch.median(loss) 
        #es_99 = np.percentile(loss.cpu().detach().numpy(), q=99)
        #es_99 = torch.mean(- torch.topk(-loss, math.ceil(0.01*30))[0])
        #es_50 = torch.mean(- torch.topk(-loss, math.ceil(0.5*30))[0])
        #zeros = torch.zeros(loss.shape[0])
        #loss_q1 = torch.max((loss-self.q1), zeros)
        es_99 = (F.relu(loss-self.q1).mean())/(1-0.99) + self.q1
        #loss_q2 = torch.max((loss-self.q2), zeros)
        es_50 = (F.relu(loss-self.q2).mean())/(1-0.5) + self.q2
        risk_measure = (es_50 + es_99*self.beta)/(1+self.beta)
        return risk_measure
'''
def loss(deltas, pay_off, growth, costs, q1, q2, beta=1):
    S_delta_T = torch.transpose(growth, 0, 1)
    delta_S = torch.diagonal(torch.matmul(deltas, S_delta_T))
    loss = torch.sub(pay_off.squeeze(), delta_S)
    #es_50 = torch.median(loss) 
    #es_99 = np.percentile(loss.cpu().detach().numpy(), q=99)
    #es_99 = torch.mean(- torch.topk(-loss, math.ceil(0.01*30))[0])
    #es_50 = torch.mean(- torch.topk(-loss, math.ceil(0.5*30))[0])
    #zeros = torch.zeros(loss.shape[0])
    #loss_q1 = torch.max((loss-self.q1), zeros)
    es_99 = (F.relu(loss-q1).mean())/(1-0.99) + q1
    #loss_q2 = torch.max((loss-self.q2), zeros)
    es_50 = (F.relu(loss-q2).mean())/(1-0.5) + q2
    risk_measure = (es_50 + es_99*beta)/(1+beta)
    return risk_measure
    

In [ ]:
num_epochs = 50
learning_rate = 0.01

input_size = 1 # number of expected features
output_size = 1
HL_size = 16
model = DeepHedging_RNN(input_size, HL_size, output_size)
# Set up optimizer
#q1 = nn.Parameter(torch.randn(1), requires_grad=True)
q1 = torch.tensor((0.), dtype=torch.float32, requires_grad=True, device=device)
q2 = torch.tensor((0.), dtype=torch.float32, requires_grad=True, device=device)
optimizer = optim.Adam(list(model.parameters()) + [q1, q2], lr=learning_rate)
#criterion = BS_Loss(batch_size = batch_size)

In [ ]:
from tqdm import tqdm

train_loss = []
val_loss = []
for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss = 0.0
    counter = 0

    for train_X, train_payoff, train_growth, costs in train_loader:
        counter += 1

        # Calculate gradients and update model weights
        optimizer.zero_grad()
        train_X = train_X.unsqueeze(-1)
        deltas = model(train_X)
        losses = loss(deltas, train_payoff, train_growth, costs, q1, q2)
        running_loss += losses.item()
        
        losses.backward()
        optimizer.step()
        
    epoch_train_loss = running_loss / counter
    train_loss.append(epoch_train_loss)
    print("Train loss: {}".format(epoch_train_loss))
    
    running_loss_test = 0.0
    counter = 0

    for val_X, val_payoff, val_growth, costs in val_loader:
        model.eval()
        
        counter += 1
        val_X = val_X.unsqueeze(-1)
        deltas = model(val_X)
        losses_test = loss(deltas, val_payoff, val_growth, costs, q1, q2)
        running_loss_test += losses_test.item()

    epoch_test_loss = running_loss_test / counter
    val_loss.append(epoch_test_loss)
    print("Test loss: {}".format(epoch_test_loss))

In [ ]:
def get_data_eval(url_data, url_payoff):
    df = import_dataset_fct.get_file(url_data)
    pay_off = pd.read_csv(url_payoff, header=None)
    df_absolute_growth = absolute_growth(df).iloc[:,1:]
    df = standardize(df.iloc[:,:30])
    data = np.array(df.values.astype('float32'))
    pay_off = np.array(pay_off)
    growth = np.array(df_absolute_growth.values.astype('float32'))
    pay_off = torch.Tensor(pay_off)
    data = torch.Tensor(data)
    growth = torch.Tensor(growth)
    return data, pay_off, growth
url_payoff = "/home/jovyan/hfactory_magic_folders/natixis_data_challenge_22_23/erm/Dataset_1_train_payoff.csv"
url_data = "Dataset_1_train_asset.csv"
data, pay_off, growth = get_data_eval(url_data, url_payoff)
data = data.to(device).unsqueeze(-1)
predictions = model(data)

In [ ]:
def evaluate(deltas, pay_off, growth, beta=1):
    S_delta_T = torch.transpose(growth, 0, 1)
    delta_S = torch.diagonal(torch.matmul(deltas, S_delta_T))
    loss = torch.sub(pay_off.squeeze(), delta_S)
    es_99 = torch.mean(- torch.topk(-loss, math.ceil(0.01*10000))[0])
    es_50 = torch.mean(- torch.topk(-loss, math.ceil(0.5*10000))[0])
    risk_measure = (es_50 + es_99*beta)/(1+beta)
    return risk_measure
evaluate(predictions, pay_off, growth)

In [ ]:
#PATH = "/home/jovyan/natives_deephedging/model_lea"
#torch.save(model.state_dict(), PATH)

In [ ]:
#model = torch.load(PATH)

In [ ]:
path_test = "/home/jovyan/natives_deephedging/data/Dataset_1_test_asset.csv"
test_set = pd.read_csv(path_test, header=None)
test_set

In [ ]:
test_X = np.array(test_set.iloc[:,:30].values.astype('float32'))
test_X = torch.Tensor(test_X)
transaction_costs_test = torch.Tensor([0.0] * test_X.shape[0])
dataset_test = torch.utils.data.TensorDataset(test_X, transaction_costs_test)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=1, shuffle=False)

In [ ]:
#test_X = np.array(test_set.iloc[:,:30].values.astype('float32'))
#test_X = torch.tensor(test_X).unsqueeze(-1)
#test_prediction = model(test_X)
#test_prediction

In [ ]:
final = []
for stock, costs in tqdm(test_loader):
    stock = stock.to(device).unsqueeze(-1)
    final.append(model(stock).cpu().detach().numpy())
final


In [ ]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [ ]:
import matplotlib.pyplot as plt
def plot_figures(rnn_bs: pd.DataFrame, df: pd.DataFrame):
    #(deltas_BS: pd.DataFrame, rnn_bs: pd.DataFrame, df: pd.DataFrame): 
    fig, axs = plt.subplots(2, 3)
    fig.set_size_inches(18.5, 10.5)
    fig.suptitle('Delta spread at different time intervals', size=45)

    #axs[0,0].plot(df.loc[:, 0], deltas_BS.loc[:, 0], 'bo', label='Black-Scholes')
    axs[0,0].plot(df.loc[:, 0], NormalizeData(rnn_bs.loc[:, 0]), 'gx', label='Delta-RNN')
    axs[0,0].legend()
    axs[0,0].set_title('Time 0')
    axs[0,0].set_ylim([0, 1])
    #axs[0,1].plot(df.loc[:, 1], deltas_BS.loc[:, 1], 'bo', label='Black-Scholes')
    axs[0,1].plot(df.loc[:, 1], NormalizeData(rnn_bs.loc[:, 1]), 'gx', label='Delta-RNN')
    axs[0,1].legend()
    axs[0,1].set_title('Time 1')
    #axs[0,2].plot(df.loc[:, 5], deltas_BS.loc[:, 5], 'bo', label='Black-Scholes')
    axs[0,2].plot(df.loc[:, 5], NormalizeData(rnn_bs.loc[:, 5]), 'gx', label='Delta-RNN')
    axs[0,2].legend()
    axs[0,2].set_title('Time 5')
    #axs[1,0].plot(df.loc[:, 15], deltas_BS.loc[:, 15], 'bo', label='Black-Scholes')
    axs[1,0].plot(df.loc[:, 15], NormalizeData(rnn_bs.loc[:, 15]), 'gx', label='Delta-RNN')
    axs[1,0].legend()
    axs[1,0].set_title('Time 15')
    #axs[1,1].plot(df.loc[:, 25], deltas_BS.loc[:, 25], 'bo', label='Black-Scholes')
    axs[1,1].plot(df.loc[:, 25], NormalizeData(rnn_bs.loc[:, 25]), 'gx', label='Delta-RNN')
    axs[1,1].legend()
    axs[1,1].set_title('Time 25')
    #axs[1,2].plot(df.loc[:, 30], deltas_BS.loc[:, 30], 'bo', label='Black-Scholes')
    axs[1,2].plot(df.loc[:, 30], NormalizeData(rnn_bs.loc[:, 25]), 'gx', label='Delta-RNN')
    axs[1,2].legend()
    axs[1,2].set_title('Time 30')

    for ax in axs.flat:
        ax.set(xlabel='Spot price', ylabel='Delta')

In [ ]:
# exercise 1 - BlackScholes Model
from scipy.stats import norm
import matplotlib.pyplot as plt
N = norm.cdf

def bs_call(K: float, #underlying asset's price
            T: float, #time till maturity
            S: float = 1.0, # strike price
            r: float = 0.0,  # risk-free
            sigma: float = 0.158):
    
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T) * N(d2), N(-d1)

def bs_put(K: float, #underlying asset's price
           T: float, #time till maturity
           S: float = 1.0, #strike price
           r: float = 0.0, # risk-free
           sigma: float = 0.158):
    
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*N(-d1)  

In [ ]:
def calculate_hedge(df: pd.DataFrame):
    deltas = pd.DataFrame(index=range(df.shape[0]),columns=range(df.shape[1]))
    
    for i in tqdm(range(0, df.shape[0])):
        short_position = 0.0
        for j in range(0, df.shape[1]):
            S = 100
            initial_price = 100
            K = df.loc[i, :].values[j]
            T = (31 - j)/365
            result, delta = bs_call(K=K, T=T, S=S)
            short_position += -result
            deltas.loc[i, j] = delta

    return deltas

In [ ]:
bs_deltas_test = calculate_hedge(test_set)

In [ ]:
#plot_figures(bs_deltas_test, pd.DataFrame(final), test_set)
plot_figures(pd.DataFrame(final), test_set)